\

### Install required dependies

In [ ]:
!pip install -q langchain \
langchain-core \
langchain-community \
langchain-openai \
langchain-groq \
langchain-google-genai \
langchain-cohere  \
pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 562.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 924.2 kB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source

### Environment checkup

In [ ]:
print('='*60)
print(f'Environment check')
print('='*60)
from google.colab import userdata
gemini_key = userdata.get('GEMINI_API_KEY')[:10]
if gemini_key:
  print(f"Gemini key Found. {gemini_key}*****")
openrouter_key = userdata.get('OPENROUTER_API_KEY')[:10]
if openrouter_key:
  print(f"Open Router key Found. {openrouter_key}*****")
groq_key = userdata.get('GROQ_API_KEY')[:10]
if groq_key:
  print(f"Groq key Found. {groq_key}*****")
print('='*60)

Environment check
Gemini key Found. AIzaSyDSmx*****
Open Router key Found. sk-or-v1-5*****
Groq key Found. gsk_zQKvLC*****


In [ ]:
import os
os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
print("Set Gemini API key in the enviornment variables")
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
print("Set Groq API key in the enviornment variables")

Set Gemini API key in the enviornment variables
Set Groq API key in the enviornment variables


In [ ]:
import os
import sys
import torch

print('='*60)
print(f'Environment check')
print('='*60)
print(f'Python Version:{sys.version}')
print(f'Device:{'Cuda' if torch.cuda.is_available() else 'CPU'}')

print("="*60)

Environment check
Python Version:3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Device:CPU


In [ ]:
from dataclasses import dataclass

@dataclass
class Config:
  data_path: str = '/content/data'
  out_dir: str = '/content/output'
  chunk_size: int = 1500
  chunk_overlap: int = 200
  qa_per_chunk: int = 10
  train_split: float = 0.8

  base_provider: str = "groq"
  provider: str = "groq"
  model: str = "llama-3.1-8b-instant"

  temperature: float = 0.2
  max_tokens: int =384
  request_timeout: int =60

In [ ]:
from pprint import pprint
config = Config()
print("="*60)
print("CONFIGURATION")
print("="*60)
pprint(config)
print("="*60)

CONFIGURATION
Config(data_path='/content/data',
       out_dir='/content/output',
       chunk_size=1500,
       chunk_overlap=200,
       qa_per_chunk=10,
       train_split=0.8,
       base_provider='groq',
       provider='groq',
       model='llama-3.1-8b-instant',
       temperature=0.2,
       max_tokens=384,
       request_timeout=60)


### Load PDF

In [ ]:
from pathlib import Path
from langchain_community.document_loaders import (
    PyPDFLoader,
    DirectoryLoader,
    TextLoader
)
pdf_dir = Path(config.data_path)
pdf_files=  list(pdf_dir.glob('*.pdf'))
if len(pdf_files) == 0:
    print(f" No PDFs found at {pdf_dir}")
else:
  documents = []
  for pdf_path in pdf_files:
    loader = PyPDFLoader(str(pdf_path))

    doc = loader.load()
    documents.extend(doc)
    print(f" Loaded {len(documents)} document pages")
    print(f" Total characters: {sum(len(d.page_content) for d in documents):,}")



 Loaded 142 document pages
 Total characters: 639,466


### Simple Cleaning

In [ ]:
import re
def clean_whitespace(text: str) -> str:
    """
    - Normalize newlines
    - Collapse multiple spaces
    - Collapse excessive blank lines
    """
    # Normalize newlines
    text = text.replace("\r\n", "\n").replace("\r", "\n")

    # Remove trailing/leading whitespace per line
    lines = [line.strip() for line in text.split("\n")]

    # Collapse multiple spaces inside lines
    lines = [re.sub(r"\s{2,}", " ", line) for line in lines]

    # Rebuild text
    text = "\n".join(lines)

    # Collapse 3+ newlines into max 2
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

In [ ]:
from langchain_core.documents import Document
cleaned_documents = []

for doc in documents:
    cleaned_text = clean_whitespace(doc.page_content)
    cleaned_documents.append(
        Document(
            page_content=cleaned_text,
            metadata=doc.metadata
        )
    )
print("Cleaned the files for chunking")

Cleaned the files for chunking


### Fixed size chunking

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

splitter = CharacterTextSplitter(
    chunk_size=config.chunk_size,
    chunk_overlap=config.chunk_overlap,
    separator=" "
)
chunks = splitter.split_documents(cleaned_documents)
for i, d in enumerate(chunks):
    d.metadata["chunk_id"] = i
print(f"No of chunks: {len(chunks)}")
print(f"\nExample chunk:")
print(f"  Length: {len(chunks[1].page_content)} chars")
print(f"  Content: {chunks[1].page_content[:150]}...")

No of chunks: 532

Example chunk:
  Length: 579 chars
  Content: Uber’s Mission
We reimagine the way the world moves for the better
We are Uber. The go-getters. The kind of people who are relentless about our
missio...


In [ ]:
def get_llm(config):
    """
    Return a LangChain-compatible chat model based on config.

    Args:
        config: Configuration dictionary with llm_provider, llm_model, etc.

    Returns:
        LangChain chat model instance

    Raises:
        ValueError: If provider is unknown
    """
    base_provider = config.base_provider

    if base_provider == "openrouter":
        openrouter_provider = config.provider
        openrouter_model = config.model
        model_name = f"{openrouter_provider}/{openrouter_model}"

        from langchain_openai import ChatOpenAI
        return ChatOpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=userdata.get("OPENROUTER_API_KEY"),
            model=model_name,
            temperature=config.temperature,
            max_tokens=config.max_tokens,
            timeout=config.request_timeout,
        )

    elif base_provider == "gemini":
        model_name = config.model
        from langchain_google_genai import ChatGoogleGenerativeAI
        return ChatGoogleGenerativeAI(
            model=model_name,
            temperature=config.temperature,
            max_tokens=config.max_tokens,
        )
    elif base_provider == "groq" :
        model_name = config.model
        from langchain_groq import ChatGroq
        return ChatGroq(
            model = model_name,
            temperature= config.temperature
        )
    else:
        raise ValueError(f"Unknown llm_provider: {provider}")

### LLM sanity check

In [ ]:
llm_a = get_llm(config)

print("\nTesting LLM API connection...")
try:
  test_chunk = chunks[100]
  print(test_chunk.metadata)
  print("Chunk ID:", test_chunk.metadata.get("chunk_id"))
  print("Page:", test_chunk.metadata.get("page_start", test_chunk.metadata.get("page")))
  print(test_chunk.page_content[:500])

  test_prompt_A =f"""
  ROLE:
  You are a senior financial analyst creating training data from an annual report.

  TASK:
  Generate questions that can be answered strictly using the provided context.

  CONTEXT:
  {test_chunk.page_content}

  CONSTRAINTS:
  - Use ONLY the information in the CONTEXT. Do not use outside knowledge.
  - Generate EXACTLY 10 questions.
  - Enforce the following category distribution exactly:
    • HardFact: 4 questions (exact figures, dates, entities, or explicitly stated facts)
    • Strategy: 3 questions (summaries, drivers, risks, or implications based on the context)
    • Style: 3 questions (stylistic or creative outputs such as memos, briefs, or tone-based summaries)
  - Questions must be specific, non-duplicative, and clearly answerable from the CONTEXT.
  - Do NOT include answers.
  - Do NOT include explanations, markdown, or any extra text.

  OUTPUT FORMAT:
  Return a valid JSON array of exactly 10 objects.
  Each object MUST have the following fields:
  - "category": one of ["HardFact", "Strategy", "Style"]
  - "question": string

  Example format:
  [
    dictionary as "category": "HardFact", "question": "...",
    ...
  ]
  """

  response = llm_a.invoke(test_prompt_A)
  raw_output = response.content
  print("*"*60)
  print("Output from the Model")
  print("*"*60)
  print(raw_output)
  print("*"*60)

except Exception as e:
    print(f"LLM API test failed: {e}")
    print("Please check your .env file and API key configuration.")




Testing LLM API connection...
{'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.0 (Macintosh)', 'creationdate': '2025-03-20T10:48:37+05:30', 'moddate': '2025-03-22T09:33:29-04:00', 'trapped': '/False', 'source': '/content/data/annual_report.pdf', 'total_pages': 142, 'page': 24, 'page_label': 'y', 'chunk_id': 100}
Chunk ID: 100
Page: 24
we are not successful in penetrating suburban and rural areas, or if we are unable to operate in certain key metropolitan ar eas in the
future, our ability to serve what we consider to be our total addressable market would be limited, and our business, financial condition,
and operating results would suffer.
In 2024, we generated 15% of our Mobility Gross Bookings from tr ips that either started or were completed at an airport. As a
result of this concentration, our operating results are suscept
************************************************************
Output from the Model
**********************************************************

### Question Generation

In [ ]:
import json
from collections import Counter

categories = {"HardFact", "Strategy", "Style"}

def parse_json_array(text: str):
    """Parse JSON array from model output (assumes prompt asked JSON-only)."""
    return json.loads(text)

def validate_questions(q_list):
    """Return (ok: bool, error: str)."""
    if not isinstance(q_list, list):
        return False, "Not a list"

    if len(q_list) != 10:
        return False, f"Expected 10 questions, got {len(q_list)}"

    for i, item in enumerate(q_list):
        if not isinstance(item, dict):
            return False, f"Item {i} not an object"
        if "category" not in item or "question" not in item:
            return False, f"Item {i} missing keys"
        if item["category"] not in categories:
            return False, f"Item {i} has invalid category: {item['category']}"
        if not isinstance(item["question"], str) or not item["question"].strip():
            return False, f"Item {i} empty question"

    counts = Counter([x["category"] for x in q_list])
    if counts["HardFact"] != 4 or counts["Strategy"] != 3 or counts["Style"] != 3:
        return False, f"Bad category distribution: {dict(counts)}"

    return True, "ok"

In [ ]:
def make_question_prompt(chunk_text: str) -> str:
    return f"""
ROLE:
You are a senior financial analyst creating training data from an annual report.

TASK:
Generate questions that can be answered strictly using the provided context.

CONTEXT:
{chunk_text}

CONSTRAINTS:
- Use ONLY the information in the CONTEXT.
- Generate EXACTLY 10 questions.
- Category distribution must be:
  • HardFact: 4
  • Strategy: 3
  • Style: 3
- Questions must be specific and non-duplicative.
- Do NOT include answers.
- Output MUST be valid JSON only. No extra text.

OUTPUT FORMAT:
Return a JSON array of exactly 10 objects:
[
  {{"category": "HardFact", "question": "..."}},
  ...
]
""".strip()

In [ ]:
import os, time
from tqdm import tqdm

MAX_RETRIES = 3
CHECKPOINT_EVERY = 10
CHECKPOINT_PATH = "/content/output/questions_checkpoint.json"

questions_by_chunk = {}
fail_log = []

# optional: resume from checkpoint
if os.path.exists(CHECKPOINT_PATH):
    with open(CHECKPOINT_PATH, "r") as f:
        questions_by_chunk = json.load(f)
    # keys might be strings after JSON load
    questions_by_chunk = {int(k): v for k, v in questions_by_chunk.items()}
    print(f"Resumed from checkpoint: {len(questions_by_chunk)} chunks loaded")

for d in tqdm(chunks):
    chunk_id = d.metadata.get("chunk_id")
    if chunk_id is None:
        continue

    # skip if already done (resume support)
    if chunk_id in questions_by_chunk:
        continue

    prompt = make_question_prompt(d.page_content)

    ok = False
    last_err = None

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            resp = llm_a.invoke(prompt).content
            q_list = parse_json_array(resp)
            valid, msg = validate_questions(q_list)
            if valid:
                questions_by_chunk[chunk_id] = q_list
                ok = True
                break
            else:
                last_err = msg
        except Exception as e:
            last_err = str(e)

        # brief backoff (helps rate limits)
        time.sleep(1.0 * attempt)

    if not ok:
        fail_log.append({"chunk_id": chunk_id, "error": last_err})
        # you can decide to continue or raise
        continue

    # checkpoint periodically
    if len(questions_by_chunk) % CHECKPOINT_EVERY == 0:
        with open(CHECKPOINT_PATH, "w") as f:
            json.dump(questions_by_chunk, f)

100%|██████████| 532/532 [58:52<00:00,  6.64s/it]


#### LLM B

In [ ]:
config_b = Config(model="meta-llama/llama-4-scout-17b-16e-instruct")
print("="*60)
print("CONFIGURATION")
print("="*60)
pprint(config_b)
print("="*60)

CONFIGURATION
Config(data_path='/content/data',
       out_dir='/content/output',
       chunk_size=1500,
       chunk_overlap=200,
       qa_per_chunk=10,
       train_split=0.8,
       base_provider='groq',
       provider='groq',
       model='meta-llama/llama-4-scout-17b-16e-instruct',
       temperature=0.2,
       max_tokens=384,
       request_timeout=60)


In [ ]:
llm_b = get_llm(config_b)
print("\nTesting LLM API connection...")
try:
  test_chunk = chunks[0]
  test_question = q_list[0]
  print(test_chunk.metadata)
  print("Chunk ID:", test_chunk.metadata.get("chunk_id"))
  print("Page:", test_chunk.metadata.get("page_start", test_chunk.metadata.get("page")))
  print(test_chunk.page_content[:500])

  test_prompt_b =f"""
  ROLE:
You are a financial analyst answering questions using an annual report excerpt.

TASK:
Answer the provided questions using only the given context.

CONTEXT:
{test_chunk}

QUESTIONS:
{test_question}

CONSTRAINTS:
- Use ONLY the information in the CONTEXT.
- Do NOT use any external knowledge or assumptions.
- If the answer is not explicitly supported by the CONTEXT, respond exactly with:
  "Not available in the provided context."
- Keep answers concise and factual.
- For numeric answers, copy figures exactly as they appear in the CONTEXT.
- Output MUST be valid JSON only. No extra text.

OUTPUT FORMAT:
Return a JSON array with the SAME length and order as the input QUESTIONS:
[
  {{"category": "...", "question": "...", "answer": "..."}},
  ...
]
  """

  response = llm_a.invoke(test_prompt_b)
  raw_output = response.content
  print("*"*60)
  print("Output from the Model")
  print("*"*60)
  print(raw_output)
  print("*"*60)

except Exception as e:
    print(f"LLM API test failed: {e}")
    print("Please check your .env file and API key configuration.")




Testing LLM API connection...
{'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 19.0 (Macintosh)', 'creationdate': '2025-03-20T10:48:37+05:30', 'moddate': '2025-03-22T09:33:29-04:00', 'trapped': '/False', 'source': '/content/data/annual_report.pdf', 'total_pages': 142, 'page': 0, 'page_label': 'a', 'chunk_id': 0}
Chunk ID: 0
Page: 0
On Our Way
2024 ANNUAL REPORT
************************************************************
Output from the Model
************************************************************
[
  {
    "category": "HardFact",
    "question": "What is the address of the company's headquarters?",
    "answer": "Not available in the provided context."
  }
]
************************************************************


In [ ]:
import json

def make_answer_prompt(chunk_text: str, questions_json: str) -> str:
    return f"""
ROLE:
You are a financial analyst answering questions using an annual report excerpt.

TASK:
Answer the provided questions using only the given context.

CONTEXT:
{chunk_text}

QUESTIONS:
{questions_json}

CONSTRAINTS:
- Use ONLY the information in the CONTEXT.
- Do NOT use any external knowledge or assumptions.
- If the answer is not explicitly supported by the CONTEXT, respond exactly with:
  "Not available in the provided context."
- Keep answers concise and factual.
- For numeric answers, copy figures exactly as they appear in the CONTEXT.
- Output MUST be valid JSON only. No extra text.

OUTPUT FORMAT:
Return a JSON array with the SAME length and order as the input QUESTIONS:
[
  {{"category": "...", "question": "...", "answer": "..."}},
  ...
]
""".strip()

In [ ]:
from collections import Counter

categories = {"HardFact", "Strategy", "Style"}

def parse_json_array(text: str):
    return json.loads(text)

def validate_answers(ans_list):
    if not isinstance(ans_list, list):
        return False, "Not a list"
    if len(ans_list) != 10:
        return False, f"Expected 10 answers, got {len(ans_list)}"

    for i, item in enumerate(ans_list):
        if not isinstance(item, dict):
            return False, f"Item {i} not an object"
        for key in ("category", "question", "answer"):
            if key not in item:
                return False, f"Item {i} missing key: {key}"
        if item["category"] not in categories:
            return False, f"Item {i} invalid category: {item['category']}"
        if not isinstance(item["question"], str) or not item["question"].strip():
            return False, f"Item {i} empty question"
        if not isinstance(item["answer"], str) or not item["answer"].strip():
            return False, f"Item {i} empty answer"

    # Optional: ensure category distribution preserved (should match questions)
    counts = Counter([x["category"] for x in ans_list])
    if counts["HardFact"] != 4 or counts["Strategy"] != 3 or counts["Style"] != 3:
        return False, f"Bad category distribution: {dict(counts)}"

    return True, "ok"

In [ ]:
import os, time, json
from tqdm import tqdm

MAX_RETRIES = 3
CHECKPOINT_EVERY = 10

ANS_CHECKPOINT_PATH = "/content/output/answers_checkpoint.json"
FAIL_LOG_PATH = "/content/output/answer_fail_log.json"

# We'll store answers by chunk_id, and also flatten into Q/A records later
answers_by_chunk = {}
fail_log = []

# Resume if checkpoint exists
if os.path.exists(ANS_CHECKPOINT_PATH):
    with open(ANS_CHECKPOINT_PATH, "r") as f:
        answers_by_chunk = json.load(f)
    answers_by_chunk = {int(k): v for k, v in answers_by_chunk.items()}
    print(f"Resumed answers checkpoint: {len(answers_by_chunk)} chunks")

# Helpful: build a quick lookup from chunk_id -> chunk Document
chunk_map = {d.metadata.get("chunk_id"): d for d in chunks}

for chunk_id, q_list in tqdm(questions_by_chunk.items()):
    if chunk_id in answers_by_chunk:
        continue

    d = chunk_map.get(chunk_id)
    if d is None:
        fail_log.append({"chunk_id": chunk_id, "error": "chunk_id not found in chunks"})
        continue

    questions_json = json.dumps(q_list, ensure_ascii=False)
    prompt = make_answer_prompt(d.page_content, questions_json)

    ok = False
    last_err = None

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            resp = llm_b.invoke(prompt).content
            ans_list = parse_json_array(resp)
            valid, msg = validate_answers(ans_list)
            if valid:
                answers_by_chunk[chunk_id] = ans_list
                ok = True
                break
            else:
                last_err = msg
        except Exception as e:
            last_err = str(e)

        time.sleep(1.0 * attempt)  # backoff

    if not ok:
        fail_log.append({"chunk_id": chunk_id, "error": last_err})
        continue

    # checkpoint periodically
    if len(answers_by_chunk) % CHECKPOINT_EVERY == 0:
        with open(ANS_CHECKPOINT_PATH, "w") as f:
            json.dump(answers_by_chunk, f, ensure_ascii=False)
        with open(FAIL_LOG_PATH, "w") as f:
            json.dump(fail_log, f, ensure_ascii=False)

# Final save
with open(ANS_CHECKPOINT_PATH, "w") as f:
    json.dump(answers_by_chunk, f, ensure_ascii=False)
with open(FAIL_LOG_PATH, "w") as f:
    json.dump(fail_log, f, ensure_ascii=False)

print("Answers generated for chunks:", len(answers_by_chunk))
print("Failures:", len(fail_log))
if fail_log:
    print("Sample failure:", fail_log[0])

Resumed answers checkpoint: 156 chunks


100%|██████████| 529/529 [20:29<00:00,  2.32s/it]

Answers generated for chunks: 520
Failures: 9
Sample failure: {'chunk_id': 317, 'error': 'Item 0 missing key: answer'}


In [ ]:
import random
import datetime

# fixed instruction for all examples (keep identical for SFT consistency)
INSTRUCTION = (
    'Answer the question using ONLY the information provided in the context. '
    'If the answer is not present in the context, reply exactly: '
    '"Not available in the provided context."'
)

# chunk_id -> chunk doc lookup
chunk_map = {d.metadata.get("chunk_id"): d for d in chunks}

records = []
missing_chunks = 0

for chunk_id, ans_list in answers_by_chunk.items():
    chunk_doc = chunk_map.get(chunk_id)
    if chunk_doc is None:
        missing_chunks += 1
        continue

    context = chunk_doc.page_content.strip()

    for qa in ans_list:
        q = qa["question"].strip()
        a = qa["answer"].strip()
        cat = qa.get("category", "Unknown")

        rec = {
            "instruction": INSTRUCTION,
            "input": f"Context:\n{context}\n\nQuestion:\n{q}",
            "output": a,
            "category": cat,
        }
        records.append(rec)

print("Total records:", len(records))
print("Missing chunks:", missing_chunks)
print("Sample record:\n", records[0])

Total records: 5200
Missing chunks: 0
Sample record:
 {'instruction': 'Answer the question using ONLY the information provided in the context. If the answer is not present in the context, reply exactly: "Not available in the provided context."', 'input': 'Context:\nOn Our Way\n2024 ANNUAL REPORT\n\nQuestion:\nWhat is the total revenue for the year 2024?', 'output': '$1,234,567', 'category': 'HardFact'}


In [ ]:
from collections import Counter

cat_counts = Counter(r["category"] for r in records)
abstain = sum(1 for r in records if r["output"] == "Not available in the provided context.")

print("Category counts:", dict(cat_counts))
print("Abstain count:", abstain, f"({abstain/len(records)*100:.2f}%)")

SEED = 42
rng = random.Random(SEED)
rng.shuffle(records)

split_idx = int(0.8 * len(records))
train_records = records[:split_idx]
golden_records = records[split_idx:]

print("Train:", len(train_records))
print("Golden:", len(golden_records))

Category counts: {'HardFact': 2080, 'Strategy': 1560, 'Style': 1560}
Abstain count: 990 (19.04%)
Train: 4160
Golden: 1040


In [ ]:
import os, json

OUT_DIR = "/content/data/processed"
os.makedirs(OUT_DIR, exist_ok=True)

train_path = os.path.join(OUT_DIR, "train.jsonl")
test_path = os.path.join(OUT_DIR, "test_set.jsonl")

def write_jsonl(path, rows):
    with open(path, "w", encoding="utf-8") as f:
        for row in rows:
            f.write(json.dumps(row, ensure_ascii=False) + "\n")

write_jsonl(train_path, train_records)
write_jsonl(test_path, golden_records)

print("Saved:", train_path)
print("Saved:", test_path)

Saved: /content/data/processed/train.jsonl
Saved: /content/data/processed/test_set.jsonl
